# 案例 1.2: 泵站多机组协调控制

本案例演示了泵站的多机组协调控制。一个智能体 (`PumpControlAgent`) 会监听流量需求，并通过开关一组水泵来满足该需求。

### 1. 导入库并加载配置

In [ ]:
import json
from swp.central_coordination.collaboration.message_bus import MessageBus
from swp.simulation_identification.physical_objects.pump import Pump, PumpStation
from swp.local_agents.control.pump_control_agent import PumpControlAgent

CONFIG_PATH = 'examples/pump_station_control.json'

with open(CONFIG_PATH, 'r') as f:
    config = json.load(f)

print("配置加载成功！")

### 2. 定义仿真函数

该函数根据配置搭建泵站、控制智能体，并模拟在不同流量需求下的响应。

In [ ]:
def run_pump_station_simulation(cfg):
    # 1. 初始化消息总线
    bus = MessageBus()

    # 2. 从配置中获取主题和参数
    topics = cfg['messaging_topics']
    demand_topic = topics['demand_topic']
    control_topic_prefix = topics['control_topic_prefix']
    station_cfg = cfg['pump_station']
    pump_params = station_cfg['pump_params']
    num_pumps = station_cfg['num_pumps']
    
    # 3. 创建水泵实例
    pumps = []
    for i in range(num_pumps):
        pump_name = f"pump_{i+1}"
        pump = Pump(
            name=pump_name,
            initial_state={'status': 0},
            parameters=pump_params,
            message_bus=bus,
            action_topic=f"{control_topic_prefix}.{pump_name}"
        )
        pumps.append(pump)

    # 4. 创建泵站来管理水泵
    pump_station = PumpStation(name="main_pump_station", initial_state={}, parameters={}, pumps=pumps)

    # 5. 创建控制智能体
    agent = PumpControlAgent(
        agent_id="PumpController",
        message_bus=bus,
        pump_station=pump_station,
        demand_topic=demand_topic,
        control_topic_prefix=control_topic_prefix
    )

    # 6. 仿真循环
    print("\n--- 开始仿真 ---")
    sim_cfg = cfg['simulation_run']
    demands_to_test = sim_cfg['demands_to_test']
    dummy_heads = sim_cfg['dummy_heads']

    for step in range(sim_cfg['num_steps']):
        print(f"\n--- 时间步 {step} ---")

        # 如果当前步有指定的流量需求，则发布消息
        step_str = str(step)
        if step_str in demands_to_test:
            demand_value = demands_to_test[step_str]
            demand_message = {'value': demand_value, 'sender': 'System'}
            bus.publish(demand_topic, demand_message)

        # 智能体根据最新需求执行控制逻辑
        agent.execute_control_logic()

        # 模拟泵站对控制信号的响应
        station_action = {'upstream_head': dummy_heads['upstream'], 'downstream_head': dummy_heads['downstream']}
        pump_station.step(station_action, dt=1)

        station_state = pump_station.get_state()
        print(f"泵站状态: {station_state['active_pumps']} 台水泵运行, "
              f"总流量: {station_state['total_outflow']:.2f} m^3/s")

### 3. 运行仿真

In [ ]:
run_pump_station_simulation(config)